In [25]:
# reflected location error in ozone data simulation

import torch
import torch.fft
import numpy as np
import sys
import os
import logging
import argparse 
import pandas as pd
import pickle
import torch.optim as optim
import copy
import time
from sklearn.neighbors import BallTree
from typing import Optional, List, Tuple
from pathlib import Path

# --- CUSTOM PATHS ---
# gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
# sys.path.append(gems_tco_path)

# (필요 시 실제 GEMS_TCO 라이브러리 import)
try:
    from GEMS_TCO import kernels_reparam_space_time_gpu
    from GEMS_TCO import orderings as _orderings
    from GEMS_TCO import alg_optimization, BaseLogger
except ImportError:
    print("Warning: GEMS_TCO modules not found. Ensure paths are correct.")

# --- 1. CONFIGURATION ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float32 if DEVICE.type == 'mps' else torch.float64
print(f"Simulating on: {DEVICE}")

# TRUE PARAMETERS
init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_advec_lat = 0.0418
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map parameters
init_phi2 = 1.0 / init_range_lon
init_phi1 = init_sigmasq * init_phi2
init_phi3 = (init_range_lon / init_range_lat)**2
init_phi4 = (init_range_lon / init_range_time)**2

initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

params_list = [torch.tensor([val], requires_grad=True, dtype=DTYPE, device=DEVICE) for val in initial_vals]
OZONE_MEAN = 260.0

# --- 2. EXACT COVARIANCE & FFT HELPERS ---

def get_model_covariance_on_grid(lags_x, lags_y, lags_t, params):
    phi1, phi2, phi3, phi4 = torch.exp(params[0]), torch.exp(params[1]), torch.exp(params[2]), torch.exp(params[3])
    advec_lat, advec_lon = params[4], params[5]
    sigmasq = phi1 / phi2

    u_lat_eff = lags_x - advec_lat * lags_t
    u_lon_eff = lags_y - advec_lon * lags_t
    dist_sq = (u_lat_eff.pow(2) * phi3) + (u_lon_eff.pow(2)) + (lags_t.pow(2) * phi4)
    return sigmasq * torch.exp(-torch.sqrt(dist_sq + 1e-8) * phi2)

def generate_exact_gems_field(lat_coords, lon_coords, t_steps, params):
    Nx, Ny, Nt = len(lat_coords), len(lon_coords), t_steps
    dlat = float(lat_coords[1] - lat_coords[0])
    dlon = float(lon_coords[1] - lon_coords[0])
    dt = 1.0 
    
    # 2x Padding logic for FFT
    Px, Py, Pt = 2*Nx, 2*Ny, 2*Nt
    
    lags_x = torch.arange(Px, device=DEVICE, dtype=DTYPE) * dlat
    lags_x[Px//2:] -= (Px * dlat) # Wrap logic
    
    lags_y = torch.arange(Py, device=DEVICE, dtype=DTYPE) * dlon
    lags_y[Py//2:] -= (Py * dlon)

    lags_t = torch.arange(Pt, device=DEVICE, dtype=DTYPE) * dt
    lags_t[Pt//2:] -= (Pt * dt)

    L_x, L_y, L_t = torch.meshgrid(lags_x, lags_y, lags_t, indexing='ij')
    C_vals = get_model_covariance_on_grid(L_x, L_y, L_t, params)

    S = torch.fft.fftn(C_vals)
    S.real = torch.clamp(S.real, min=0)
    
    # Random phase
    random_phase = torch.fft.fftn(torch.randn(Px, Py, Pt, device=DEVICE, dtype=DTYPE))
    field_sim = torch.fft.ifftn(torch.sqrt(S.real) * random_phase).real
    
    return field_sim[:Nx, :Ny, :Nt]

# --- 3. GRID & MAPPING FUNCTIONS ---

def make_target_grid(lat_start, lat_end, lat_step, lon_start, lon_end, lon_step, device, dtype):
    lats = torch.arange(lat_start, lat_end - 0.0001, lat_step, device=device, dtype=dtype)
    lats = torch.round(lats * 10000) / 10000
    lons = torch.arange(lon_start, lon_end - 0.0001, lon_step, device=device, dtype=dtype)
    lons = torch.round(lons * 10000) / 10000
    grid_lat, grid_lon = torch.meshgrid(lats, lons, indexing='ij')
    center_points = torch.stack([grid_lat.flatten(), grid_lon.flatten()], dim=1)
    return center_points, len(lats), len(lons)

def coarse_by_center_tensor(
    input_map_tensors: dict, 
    target_grid_tensor: torch.Tensor, 
    use_regular_coords: bool = True 
):
    """
    Maps irregular data to target grid.
    use_regular_coords=True : Output [RegLat, RegLon, Val, Time, IrregLat, IrregLon]
    use_regular_coords=False: Output [IrregLat, IrregLon, Val, Time, RegLat, RegLon]
    """
    coarse_map = {}
    query_points_np = target_grid_tensor.cpu().numpy()
    query_points_rad = np.radians(query_points_np)
    
    for key, val_tensor in input_map_tensors.items():
        source_locs_np = val_tensor[:, :2].cpu().numpy()
        source_locs_rad = np.radians(source_locs_np)
        
        # 1. Find Nearest Irregular Point for each Grid Point
        tree = BallTree(source_locs_rad, metric='haversine')
        _, ind = tree.query(query_points_rad, k=1)
        nearest_indices = ind.flatten()
        indices_tensor = torch.tensor(nearest_indices, device=val_tensor.device, dtype=torch.long)
        
        # 2. Extract Data
        val = val_tensor[indices_tensor, 2]
        time = val_tensor[indices_tensor, 3]
        
        # 3. Coordinate Swapping Logic
        # (A) Regular Coordinates (from Target Grid)
        reg_lat, reg_lon = target_grid_tensor[:, 0], target_grid_tensor[:, 1]
        # (B) Irregular Coordinates (from Mapped Source)
        irreg_lat, irreg_lon = val_tensor[indices_tensor, 0], val_tensor[indices_tensor, 1]
        
        if use_regular_coords:
            # Main: Regular / Aux: Irregular
            cols = [reg_lat, reg_lon, val, time, irreg_lat, irreg_lon]
        else:
            # Main: Irregular / Aux: Regular
            cols = [irreg_lat, irreg_lon, val, time, reg_lat, reg_lon]

        coarse_map[key] = torch.stack(cols, dim=1)

    return coarse_map

# --- 4. ORDERING HELPER (Updated) ---

def get_spatial_ordering(
        input_maps: dict,
        mm_cond_number: int = 10,
        coord_cols: Tuple[int, int] = (0, 1) # [NEW] Column indices to use for ordering
    ) -> Tuple[np.ndarray, list]:
        
        key_list = list(input_maps.keys())
        data_for_coord = input_maps[key_list[0]]
        
        if isinstance(data_for_coord, torch.Tensor):
            data_for_coord = data_for_coord.cpu().numpy()

        # [NEW] Use specified columns (Regular Grid) for robust ordering
        x1 = data_for_coord[:, coord_cols[0]]
        y1 = data_for_coord[:, coord_cols[1]]
        coords1 = np.stack((x1, y1), axis=-1)

        # 1. MaxMin Ordering
        ord_mm = _orderings.maxmin_cpp(coords1)
        
        # 2. NN Search (based on reordered regular coords)
        coords1_reordered = coords1[ord_mm]
        nns_map_dict = _orderings.find_nns_l2(locs=coords1_reordered, max_nn=mm_cond_number)
        nns_map_list = [nns_map_dict[i] for i in range(len(nns_map_dict))]
        
        return ord_mm, nns_map_list




Simulating on: cpu


In [34]:
# --- 5. EXECUTION & MAIN PIPELINE (Scenario 2: High-Res & Random Sampling) ---

# ==============================================================================
# [CONTROLS]
# True : 관측값을 격자 중앙값으로 가정 (Random Sampling으로 인한 Noise Bias 발생)
# False: 관측값의 실제 위치 사용 (정확한 위치)
USE_REGULAR_MAIN = True 
# ==============================================================================

# 1. Target Grid 설정
target_lat_start, target_lat_end = 2.0, -3.0   
target_lon_start, target_lon_end = 121.0, 131.0
step_lat = 0.044 
step_lon = 0.063

# 2. High-Resolution Truth 생성 (4배 촘촘하게 -> 1픽셀 안에 16개 서브픽셀 존재)
UPSCALE = 4  
sim_step_lat = step_lat / UPSCALE
sim_step_lon = step_lon / UPSCALE

lat_span = abs(target_lat_start - target_lat_end)
lon_span = abs(target_lon_start - target_lon_end)
expansion_factor = 1.1 
lat_padding = lat_span * 0.1
lon_padding = lon_span * 0.1

# High-Res 격자 생성
lats_high_res = torch.arange(
    target_lat_start + lat_padding, 
    target_lat_end - lat_padding - 0.001, 
    -sim_step_lat, device=DEVICE, dtype=DTYPE
)
lons_high_res = torch.arange(
    target_lon_start - lon_padding, 
    target_lon_end + lon_padding + 0.001, 
    sim_step_lon, device=DEVICE, dtype=DTYPE
)

print(f"1. Generating High-Res Truth ({UPSCALE}x finer)...")
t_def = 8
# High-Res Field 생성 (이게 진짜 세상)
sim_field_high_res = generate_exact_gems_field(lats_high_res, lons_high_res, t_def, params_list)


# 3. Sampling: "Random Sub-pixel Sampling" (주변에서 랜덤으로 선택)
print("2. Sampling Irregular Data (Random Sub-pixel)...")

# Target Grid 생성
target_grid, Nx_reg, Ny_reg = make_target_grid(
    target_lat_start, target_lat_end, -step_lat, 
    target_lon_start, target_lon_end, step_lon,  
    DEVICE, DTYPE
)

# High-Res 좌표계와 Target Grid 간의 관계 찾기
mesh_lat_hr, mesh_lon_hr = torch.meshgrid(lats_high_res, lons_high_res, indexing='ij')
flat_lats_hr = mesh_lat_hr.flatten()
flat_lons_hr = mesh_lon_hr.flatten()

coords_hr_np = torch.stack([flat_lats_hr, flat_lons_hr], dim=1).cpu().numpy()
target_coords_np = target_grid.cpu().numpy()

tree = BallTree(np.radians(coords_hr_np), metric='haversine')
# 각 Target Point 주변의 세부 픽셀 16개(4x4)를 모두 찾습니다.
dists, indices = tree.query(np.radians(target_coords_np), k=UPSCALE**2) 

raw_extended_map = {}
nugget_std = torch.sqrt(torch.exp(params_list[6]))

for t in range(t_def):
    field_t_hr = sim_field_high_res[:, :, t].flatten()
    
    # [핵심 수정] 16개(UPSCALE^2)의 주변 픽셀 중 "랜덤하게 하나"를 뽑습니다.
    # indices shape: [Num_Target_Points, 16]
    # 각 행마다 0~15 사이의 랜덤 인덱스 생성
    random_neighbor_idx = torch.randint(0, UPSCALE**2, (indices.shape[0],))
    
    # Numpy Indexing을 위해 행 번호 생성
    row_indices = np.arange(indices.shape[0])
    
    # 최종 선택된 High-Res 인덱스들
    chosen_indices = indices[row_indices, random_neighbor_idx.numpy()]
    chosen_indices_tensor = torch.tensor(chosen_indices, device=DEVICE, dtype=torch.long)
    
    # 1. 그 랜덤 위치의 "진짜 값"과 "진짜 좌표" 추출
    # (주의: 노이즈를 더해서 위치를 흔드는게 아니라, 진짜 존재하는 다른 위치를 가져오는 것)
    true_vals = field_t_hr[chosen_indices_tensor]
    true_lats = flat_lats_hr[chosen_indices_tensor]
    true_lons = flat_lons_hr[chosen_indices_tensor]
    
    # 2. 관측 노이즈 추가
    obs_vals = true_vals + (torch.randn_like(true_vals) * nugget_std) + OZONE_MEAN
    
    # 3. 데이터 저장
    row_tensor = torch.stack([
        true_lats,  
        true_lons,  
        obs_vals,               
        torch.full_like(true_lats, 21.0 + t)
    ], dim=1)
    
    key_str = f'2024_07_y24m07day01_hm{t:02d}:53'
    raw_extended_map[key_str] = row_tensor.detach()


# 4. Mapping & Results
print("\n--- Processing to Target Grid ---")

inputmap = coarse_by_center_tensor(
    raw_extended_map, 
    target_grid, 
    use_regular_coords=USE_REGULAR_MAIN
)

# 오더링은 항상 Regular Grid 기준
ordering_cols = (0, 1) if USE_REGULAR_MAIN else (4, 5)

print(f"Mode: {'Regular (Biased)' if USE_REGULAR_MAIN else 'Irregular (True)'}")

ord_mm, nns_map = get_spatial_ordering(
    inputmap, 
    mm_cond_number=15, 
    coord_cols=ordering_cols
)

mm_input_map = {}
for key in inputmap:
    mm_input_map[key] = inputmap[key][ord_mm][:, :4]

# --- 검증 ---
first_key = list(mm_input_map.keys())[0]
print(f"First row coords: {mm_input_map[first_key][0, :2].cpu().numpy()}")

1. Generating High-Res Truth (4x finer)...
2. Sampling Irregular Data (Random Sub-pixel)...

--- Processing to Target Grid ---
Mode: Regular (Biased)
First row coords: [ -0.464 125.977]


정말 예리하고 정확한 지적입니다. 님께서 느끼신 그 "찜찜함"이 맞습니다.

현재 방식대로라면 **"원래 데이터가 레귤러 그리드에 있는데, 좌표만 잘못 기록된 상황(GPS 오차 등)"**을 시뮬레이션하는 꼴이 됩니다.

만약 연구 주제가 **"실제로는 이레귤러한 위치(랜덤 위치)에서 관측된 데이터를 억지로 레귤러 그리드로 옮길 때 발생하는 바이어스(Representation Error)"**를 보는 것이라면, 현재 방식은 문제가 있습니다.

왜 그런지, 그리고 어떻게 고쳐야 하는지 설명해 드릴게요.

지만, 코드를 완전히 뒤엎기 힘들다면 가장 간단한 타협점은 다음과 같습니다.

"현재 로직을 유지하되, FFT 격자와 분석(Target) 격자를 서로 어긋나게(Misaligned) 만들기"

지금은 sim_field 생성 그리드와 target_grid가 사실상 일치(확장만 된 상태)합니다. 이를 방지하기 위해:

FFT 생성 시: step을 아주 미세하게 다르게 하거나(예: 0.044 대신 0.033), 시작점(offset)을 반 픽셀 정도 밀어버리세요.

그러면 coarse_by_center_tensor 할 때, 원래 생성된 위치(X 
sim
​	
 )가 타겟 그리드(X 
target
​	
 )와 딱 맞아떨어지지 않게 됩니다.

이렇게 하면 레귤러화할 때 필연적으로 **위치 오차(Quantization Error)**가 발생하여 바이어스를 관찰할 수 있습니다.

요약

현재 코드: 레귤러 그리드에 유리하게 편향됨 (Bias favor to Regular Grid).

이유: 생성 위치 = 타겟 위치 구조라서, 레귤러화가 곧 "정답 찾기"가 됨.

추천:

(Best) High-Resolution FFT 생성 후 랜덤 샘플링.

(Easy) FFT 생성 그리드의 간격(Step)이나 원점(Origin)을 타겟 그리드와 애매하게 어긋나게 설정.

set up

Likelihood vs. Truth: A model with the wrong parameters (short range) might produce a higher Vecchia likelihood because it fits the high-frequency noise better, but it will be terrible at prediction (Kriging) away from data points.

# Fit vecchia max min time 2 

vecc 

In [35]:
import torch
import numpy as np
import time

# --- CONFIGURATION ---
v = 0.5              # Smoothness
mm_cond_number = 8   # Neighbors
#mm_cond_number = 16   # Neighbors
nheads = 300           # 0 = Pure Vecchia
lr = 1.0             # LBFGS learning rate
LBFGS_MAX_STEPS = 3
LBFGS_HISTORY_SIZE = 100 # 100
LBFGS_LR = 1.0
LBFGS_MAX_EVAL = 30    

#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- 1. SETUP PARAMETERS (List of Scalars) ---
# Truth: [4.18, 1.94, 0.24, -3.97, 0.014, -0.20, -0.85]
init_sigmasq   = 13.059
init_range_lat = 0.154 
init_range_lon = 0.195
init_advec_lat = 0.0218
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map model parameters to the 'phi' reparameterization
init_phi2 = 1.0 / init_range_lon                # 1/range_lon
init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

# Create Initial Parameters (Float64, Requires Grad)
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

# [4.2042, 1.6348, 0.4721, -3.2695, 0.0218, -0.1689, -1.3984]
params_list = [
    torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
    for val in initial_vals
]

# --- 2. INSTANTIATE MODEL ---
print(f'\n{"="*40}')
print(f'--- Initializing VecchiaBatched Model ---')
print(f'{"="*40}')

if isinstance(aggregated_data, torch.Tensor):
    aggregated_data = aggregated_data.to(DEVICE)

# Instantiate
model_instance = kernels_reparam_space_time_gpu.fit_vecchia_lbfgs(
    smooth=v,
    input_map=mm_input_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number,
    nheads=nheads
)

# --- 3. OPTIMIZATION LOOP ---
print(f'\n{"="*40}')
print(f'--- Running L-BFGS Optimization ---')
print(f'{"="*40}')

# Optimizer takes the LIST of scalars
optimizer_vecc = model_instance.set_optimizer(
            params_list,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        
            history_size=LBFGS_HISTORY_SIZE 
        )

start_time = time.time()

out, steps_ran = model_instance.fit_vecc_lbfgs(
        params_list,
        optimizer_vecc,
        # covariance_function argument is GONE
        max_steps=LBFGS_MAX_STEPS, 
        grad_tol=1e-7
    )


end_time = time.time()
epoch_time = end_time - start_time

print(f"\nOptimization finished in {epoch_time:.2f}s.")
print(f"Results after {steps_ran} steps: {out}")
print(f"Final Params: {torch.cat(params_list).detach().cpu().numpy()}")

Using device: cpu

--- Initializing VecchiaBatched Model ---

--- Running L-BFGS Optimization ---
🚀 Pre-computing (Corrected Vectorization)... ✅ Done in 0.9453s. (Heads: 2400, Tails: 142608)
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/3 / Loss: 1.306560 ---
  Param 0: Value=4.3502, Grad=-2.0021551137707714e-05
  Param 1: Value=1.8276, Grad=1.627321658716148e-05
  Param 2: Value=0.3074, Grad=-5.732726197998198e-06
  Param 3: Value=-0.9246, Grad=2.46056435509749e-07
  Param 4: Value=0.2471, Grad=-8.056858741892475e-06
  Param 5: Value=-0.9583, Grad=-3.043290814638529e-06
  Param 6: Value=-1.0754, Grad=-8.324182970408054e-07
  Max Abs Grad: 2.002155e-05
------------------------------
--- Step 2/3 / Loss: 1.299139 ---
  Param 0: Value=4.3500, Grad=5.221495942024017e-07
  Param 1: Value=1.8271, Grad=-1.585497809935112e-08
  Param 2: Value=0.3075, Grad=1.771418075318106e-07
  Param 3: Value=-0.9324, Grad=-5.080534313694756e-10
  Param 4: Value=0.3412, Grad=-4.32606536715247

###

Final Interpretable Params: {
   'sigma_sq': 12.982146243206834, '
   range_lon': 0.1774660571793038, 
   'range_lat': 0.1377164149996857, 
   'range_time': 0.8960320754550594, 
   'advec_lat': 0.03668090767382201, 
   'advec_lon': -0.16896621297828324, 
   'nugget': 3.0318146866074696e-14}


Final Interpretable Params: {
   'sigma_sq': 12.652739121404919, 
   'range_lon': 0.17664470485305778, 
   'range_lat': 0.14565426376210802, 
   'range_time': 0.8720031476394872, 
   'advec_lat': 0.04466510371597738, 
   'advec_lon': -0.16447983825003892, 'nugget': 0.31139273714919946}



###

Final Interpretable Params: {
   'sigma_sq': 12.600402314947898, 
   'range_lon': 0.15632064225879247, 
   'range_lat': 0.12850412363793687, 
   'range_time': 0.420237386856305, 
   'advec_lat': 3.646629197697307, 
   'advec_lon': -1.82731716498955, 
   'nugget': 3.3125395377840276e-12}


   Final Interpretable Params: {
      'sigma_sq': 12.465255157603531,
       'range_lon': 0.16088131264500707, 
       'range_lat': 0.13795064901185095, 
       'range_time': 0.25642913066711404, 
       'advec_lat': 0.34121830847670426, 
       'advec_lon': -0.9086603984853497, 'nugget': 0.3413704158367853}




# TRUE PARAMETERS
init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_advec_lat = 0.0418
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247


irregular grid

Final Interpretable Params: {
   'sigma_sq': 13.1581147614216, 
   'range_lon': 0.1670857937279293, 
   'range_lat': 0.13532624419142394, 
   'range_time': 0.662083817877393, 
   'advec_lat': 4.485362408040999, 
   'advec_lon': -0.05054465623325784, 
   'nugget': 3.202037263471046e-05}

regular grid

Final Interpretable Params: {
   'sigma_sq': 13.547646279613172, 
   'range_lon': 0.2063608511650159, 
   'range_lat': 0.1698378442441089, 
   'range_time': 1.122662499886297, 
   'advec_lat': 0.05415059834603563, 
   'advec_lon': -0.16640862287821778, 
   'nugget': 0.48795387825462566}




# fit dw

difference data

In [25]:
a = [11.0474, 0.0623, 0.2445, 1.0972, 0.0101, -0.1671, 1.1825]
day = 0 # 0 index
lat_range= [0,5]
lon_range= [123.0, 133.0]
#lat_range= [1,3]
#lon_range= [125, 129.0]

daily_aggregated_tensors_dw = [aggregated_data]
daily_hourly_maps_dw = [input_map]

db = debiased_whittle.debiased_whittle_preprocess(daily_aggregated_tensors_dw, daily_hourly_maps_dw, day_idx=day, params_list=a, lat_range=lat_range, lon_range=lon_range)


subsetted_aggregated_day = db.generate_spatially_filtered_days(0,5,123,133)
print(subsetted_aggregated_day.shape)
N2= subsetted_aggregated_day.shape[0]
print(N2)
subsetted_aggregated_day[:20]

torch.Size([142832, 4])
142832


tensor([[ 2.8000e-02,  1.2305e+02,  0.0000e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2311e+02,  1.9113e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2317e+02,  3.1846e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2323e+02,  9.3044e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2330e+02,  7.5384e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2336e+02, -8.2040e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2342e+02, -1.2852e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2349e+02, -3.2619e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2355e+02, -6.2629e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2361e+02,  1.3124e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2368e+02,  4.2001e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2374e+02, -4.3216e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2380e+02, -5.5636e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2386e+02, -2.0360e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2393e+02,  4.5638e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2399e+02, -5.2148e-02,  2.1000e+01],
        

In [26]:

dwl = debiased_whittle.debiased_whittle_likelihood()
if __name__ == '__main__':
    start_time = time.time()

    # --- Configuration ---
    DAY_TO_RUN = 3 # data is decided above
    TAPERING_FUNC = dwl.cgn_hamming # Use Hamming taper
    NUM_RUNS = 1
    MAX_STEPS = 20 # L-BFGS usually converges in far fewer steps
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")

    DELTA_LAT, DELTA_LON = 0.044, 0.063 

    LAT_COL, LON_COL = 0, 1
    VAL_COL = 2 # Spatially differenced value
    TIME_COL = 3


    cur_df =subsetted_aggregated_day
    
    if cur_df.numel() == 0 or cur_df.shape[1] <= max(LAT_COL, LON_COL, VAL_COL, TIME_COL):
        print(f"Error: Data for Day {DAY_TO_RUN} is empty or invalid.")
        exit()

    unique_times = torch.unique(cur_df[:, TIME_COL])
    time_slices_list = [cur_df[cur_df[:, TIME_COL] == t_val] for t_val in unique_times]

    # --- 1. Pre-compute J-vector, Taper Grid, and Taper Autocorrelation ---
    print("Pre-computing J-vector (Hamming taper)...")
    
    # --- 💥 REVISED: Renamed 'p' to 'p_time' 💥 ---
    J_vec, n1, n2, p_time, taper_grid = dwl.generate_Jvector_tapered( 
        time_slices_list,
        tapering_func=TAPERING_FUNC, 
        lat_col=LAT_COL, lon_col=LON_COL, val_col=VAL_COL,
        device=DEVICE
    )

    if J_vec is None or J_vec.numel() == 0 or n1 == 0 or n2 == 0 or p_time == 0:
       print(f"Error: J-vector generation failed for Day {DAY_TO_RUN}.")
       exit()
       
    print("Pre-computing sample periodogram...")
    I_sample = dwl.calculate_sample_periodogram_vectorized(J_vec)

    print("Pre-computing Hamming taper autocorrelation...")
    taper_autocorr_grid = dwl.calculate_taper_autocorrelation_fft(taper_grid, n1, n2, DEVICE)

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Error: NaN/Inf in sample periodogram.")
        exit()
    if torch.isnan(taper_autocorr_grid).any() or torch.isinf(taper_autocorr_grid).any():
        print("Error: NaN/Inf in taper autocorrelation.")
        exit()

    print(f"Data grid: {n1}x{n2}, {p_time} time points. J-vector, Periodogram, Taper Autocorr on {DEVICE}.")
    # --- END REVISION ---

    # --- 2. Optimization Loop ---
    all_final_results = []
    all_final_losses = []

    for i in range(NUM_RUNS):
        print(f"\n{'='*30} Initialization Run {i+1}/{NUM_RUNS} {'='*30}")

        # --- 7-PARAMETER initialization ---
        ''' 
        init_sigmasq   = 15.0
        init_range_lat = 0.66 
        init_range_lon = 0.7 
        init_nugget    = 1.5
        init_beta      = 0.1  # Temporal range ratio
        init_advec_lat = 0.02
        init_advec_lon = -0.08
        '''
        init_sigmasq   = 13.059
        init_range_lat = 0.154 
        init_range_lon = 0.195
        init_advec_lat = 0.0218
        init_range_time = 0.7
        init_advec_lon = -0.1689
        init_nugget    = 0.247

        init_phi2 = 1.0 / init_range_lon
        init_phi1 = init_sigmasq * init_phi2
        init_phi3 = (init_range_lon / init_range_lat)**2
        # Change needed to match the spatial-temporal distance formula:
        init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

        initial_params_values = [
            np.log(init_phi1),    # [0] log_phi1
            np.log(init_phi2),    # [1] log_phi2
            np.log(init_phi3),    # [2] log_phi3
            np.log(init_phi4),    # [3] log_phi4
            init_advec_lat,       # [4] advec_lat (NOT log)
            init_advec_lon,       # [5] advec_lon (NOT log)
            np.log(init_nugget)   # [6] log_nugget
        ]
        
        print(f"Starting with FIXED params (raw log-scale): {[round(p, 4) for p in initial_params_values]}")

        params_list = [
            Parameter(torch.tensor([val], dtype=torch.float64))
            for val in initial_params_values
        ]

        # Helper to define the boundary globally for clarity
        NUGGET_LOWER_BOUND = 0.05
        LOG_NUGGET_LOWER_BOUND = np.log(NUGGET_LOWER_BOUND) # Approx -2.9957

        # --- 💥 REVISED: Use L-BFGS Optimizer 💥 ---
        optimizer = torch.optim.LBFGS(
            params_list,
            lr=1.0,           # Initial step length for line search
            max_iter=20,      # Iterations per step
            history_size=100,
            line_search_fn="strong_wolfe", # Often more robust
            tolerance_grad=1e-5
        )
        # --- END REVISION ---

        print(f"Starting optimization run {i+1} on device {DEVICE} (Hamming, 7-param ST kernel, L-BFGS)...")
        
        # --- 💥 REVISED: Call L-BFGS trainer, pass p_time 💥 ---
        nat_params_str, phi_params_str, raw_params_str, loss, steps_run = dwl.run_lbfgs_tapered(
            params_list=params_list,
            optimizer=optimizer,
            I_sample=I_sample,
            n1=n1, n2=n2, p_time=p_time,
            taper_autocorr_grid=taper_autocorr_grid, 
            max_steps=MAX_STEPS,
            device=DEVICE
        )
        # --- END REVISION ---
        
        if loss is not None:
            all_final_results.append((nat_params_str, phi_params_str, raw_params_str))
            all_final_losses.append(loss)
        else:
            all_final_losses.append(float('inf'))

    print(f"\n\n{'='*25} Overall Result from Run {'='*25} {'='*25}")
    
    valid_losses = [l for l in all_final_losses if l is not None and l != float('inf')]

    if not valid_losses:
        print(f"The run failed or resulted in an invalid loss for Day {DAY_TO_RUN}.")
    else:
        best_loss = min(valid_losses)
        best_run_index = all_final_losses.index(best_loss)
        best_results = all_final_results[best_run_index]
        
        print(f"Best Run Loss: {best_loss} (after {steps_run} steps)")
        print(f"Final Parameters (Natural Scale): {best_results[0]}")
        print(f"Final Parameters (Phi Scale)    : {best_results[1]}")
        print(f"Final Parameters (Raw Log Scale): {best_results[2]}")

    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Using device: cpu
Pre-computing J-vector (Hamming taper)...
Pre-computing sample periodogram...
Pre-computing Hamming taper autocorrelation...
Data grid: 113x158, 8 time points. J-vector, Periodogram, Taper Autocorr on cpu.

============================== Initialization Run 1/1 ==============================
Starting with FIXED params (raw log-scale): [4.2042, 1.6348, 0.4721, -2.5562, 0.0218, -0.1689, -1.3984]
Starting optimization run 1 on device cpu (Hamming, 7-param ST kernel, L-BFGS)...
--- Step 1/20 ---
 Loss: 1.906929 | Max Grad: 6.274846e-04
  Params (Raw Log): log_phi1: 4.3182, log_phi2: 1.7745, log_phi3: 0.0776, log_phi4: -3.5323, advec_lat: 0.0363, advec_lon: -0.1510, log_nugget: -1.0186
--- Step 2/20 ---
 Loss: 1.817385 | Max Grad: 4.095367e-05
  Params (Raw Log): log_phi1: 4.3182, log_phi2: 1.7744, log_phi3: 0.0775, log_phi4: -3.5320, advec_lat: 0.0363, advec_lon: -0.1510, log_nugget: -1.0177
--- Step 3/20 ---
 Loss: 1.817385 | Max Grad: 4.095367e-05
  Params (Raw Log): log

init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_advec_lat = 0.0418
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

1 st simulation (1 vs 3)

Final Interpretable Params: {'sigma_sq': 12.939260896610579, 'range_lon': 0.17242543281507933, 'range_lat': 0.1631231346200311, 'range_time': 1.1358209227858689, 'advec_lat': 0.045119029109988974, 'advec_lon': -0.17809677784942035, 'nugget': 0.3009582276680578}

Final Parameters (Natural Scale): sigmasq: 13.2415, range_lat: 0.1685, range_lon: 0.1739, range_time: 0.9681, advec_lat: 0.0395, advec_lon: -0.1732, nugget: 0.3216

2nd simulation (1 vs 3)

Final Interpretable Params: {'sigma_sq': 12.765293287952144, 'range_lon': 0.17039368470743024, 'range_lat': 0.16152132799710625, 'range_time': 1.081124889959751, 'advec_lat': 0.04635511983762563, 'advec_lon': -0.1775715292039452, 'nugget': 0.31503884896742074}


sigmasq: 13.2415, range_lat: 0.1685, range_lon: 0.1739, range_time: 0.9681, advec_lat: 0.0395, advec_lon: -0.1732, nugget: 0.3216

3nd simulation (mm 15 two times)

Final Interpretable Params: {'sigma_sq': 12.292570063933866, 'range_lon': 0.16170186578939172, 'range_lat': 0.1537053192245325, 'range_time': 0.9671689003322674, 'advec_lat': 0.04155171168950867, 'advec_lon': -0.16218073366456207, 'nugget': 0.29945872751676345}

Final Parameters (Natural Scale): sigmasq: 12.7277, range_lat: 0.1631, range_lon: 0.1696, range_time: 0.9917, advec_lat: 0.0363, advec_lon: -0.1510, nugget: 0.3614
